In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import xgboost as xgb

# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')

# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../features/*.pkl')):
    if '3_feat' in feat:
        continue
    print("'{}',".format(feat))
    
    
    

'../features/fasttext_cnn2d_5_feat.pkl',
'../features/fasttext_cnn_gru_5_feat.pkl',
'../features/fasttext_cnn_v1_5_feat.pkl',
'../features/fasttext_cnn_v2_5_feat.pkl',
'../features/fasttext_cudnn_gru_5_feat.pkl',
'../features/fasttext_gru_v1_5_feat.pkl',
'../features/fasttext_lstm_v1_5_feat.pkl',
'../features/glove_cnn2d_5_feat.pkl',
'../features/glove_cnn_gru_5_feat.pkl',
'../features/glove_cnn_v1_5_feat.pkl',
'../features/glove_cnn_v2_5_feat.pkl',
'../features/glove_cudnn_gru_5_feat.pkl',
'../features/glove_gru_v1_5_feat.pkl',
'../features/glove_lstm_v1_5_feat.pkl',
'../features/lgb1_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_attention_fasttext_10_feat.pkl',
'../features/lstm_attention_fasttext_4_feat.pkl',
'../features/lstm_attention_fasttext_adj2_4_feat.pkl',
'../features/lstm_attention_glove_5_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/muse_cnn2d_5_feat.pkl',
'../features/muse_cnn_gru_5_feat.pkl',
'..

In [2]:
feats_files = [
#'../features/fasttext_cnn2d_4_feat.pkl',
# '../features/fasttext_cnn_gru_4_feat.pkl',
'../features/fasttext_cnn_v1_4_feat.pkl',
'../features/fasttext_cnn_v2_4_feat.pkl',
# '../features/fasttext_cudnn_gru_4_feat.pkl',
# '../features/fasttext_gru_v1_4_feat.pkl',
# '../features/fasttext_lstm_v1_4_feat.pkl',
# '../features/glove_cnn2d_4_feat.pkl',
# '../features/glove_cnn_gru_4_feat.pkl',
# '../features/glove_cnn_v1_4_feat.pkl',
'../features/glove_cnn_v2_4_feat.pkl',
# '../features/glove_cudnn_gru_4_feat.pkl',
# '../features/glove_gru_v1_4_feat.pkl',
# '../features/glove_lstm_v1_4_feat.pkl',
'../features/lr_feat1.pkl',
'../features/lr_feat2.pkl',
'../features/lstm_attention_fasttext_10_feat.pkl',
# '../features/lstm_attention_fasttext_4_feat.pkl',
# '../features/lstm_attention_fasttext_adj2_4_feat.pkl',
# '../features/lstm_attention_glove_4_feat.pkl',
'../features/mnb_feat1.pkl',
'../features/mnb_feat2.pkl',
'../features/muse_cnn2d_4_feat.pkl',
# '../features/muse_cnn_gru_4_feat.pkl',
# '../features/muse_cnn_v1_4_feat.pkl',
# '../features/muse_cnn_v2_4_feat.pkl',
# '../features/muse_cudnn_gru_4_feat.pkl',
# '../features/muse_gru_v1_4_feat.pkl',
# '../features/muse_lstm_v1_4_feat.pkl',
'../features/no_pretrained_cnn2d_4_feat.pkl',
'../features/no_pretrained_cnn_gru_4_feat.pkl',
'../features/no_pretrained_cnn_v1_4_feat.pkl',
'../features/no_pretrained_cnn_v2_4_feat.pkl',
'../features/no_pretrained_cudnn_gru_4_feat.pkl',
'../features/no_pretrained_gru_v1_4_feat.pkl',
'../features/no_pretrained_lstm_v1_4_feat.pkl',
'../features/other_feat.pkl',
# '../features/pool_gru_fasttext_4_feat.pkl',
'../features/pool_gru_fasttext_adj1_10_feat.pkl',
'../features/pool_gru_fasttext_adj2_4_feat.pkl',
# '../features/pool_gru_glove_4_feat.pkl',
# '../features/tfidf_feat1.pkl',
# '../features/tfidf_feat2.pkl',
]

for feat in feats_files:
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)
    
# load y
train = pd.read_csv("../input/train.csv")
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_y = train[list_classes].values.astype('int')
print(train_x.shape)

(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 36) (153164, 36)
(159571, 6) (153164, 6)
(159571, 6) (153164, 6)
(159571, 144)
(159571, 144)


In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

def simple_ens(model_name,k=3,rnd=233):
    kf = KFold(n_splits=k, shuffle=True, random_state=rnd)
    test_pred = np.zeros((153164,6))
    all_train_loss_l,all_val_loss_l = 0,0
    all_train_auc_l,all_val_auc_l = 0,0
    
    for train_index, test_index in kf.split(train_x):
        # x,y
        curr_x,curr_y = train_x[train_index],train_y[train_index]
        hold_out_x,hold_out_y = train_x[test_index],train_y[test_index]
        val_loss_l,train_loss_l = 0,0
        val_auc_l,train_auc_l = 0,0
        d_test = xgb.DMatrix(test_x)
        
        # share params
        params = {
                'subsample': 0.9,
                'eta': 0.05,
                'max_depth': 3,
                'eval_metric':'logloss',
                #'eval_metric':'auc',
                'objective':'binary:logistic',
                #'scale_pos_weight':0.9,
                'colsample_bylevel':0.9,
                'colsample_bytree':0.9,
            
                }
        
        # train for each class
        for i in range(6):
            d_train = xgb.DMatrix(curr_x, curr_y[:,i])
            d_valid = xgb.DMatrix(hold_out_x, hold_out_y[:,i])
            watchlist = [(d_train, 'train'), (d_valid, 'valid')]

            model = xgb.train(params, d_train, 1000, watchlist,
                              early_stopping_rounds=50,
                              verbose_eval=None)
            print(i)
            try:
                train_pred = model.predict(d_train)
                tmp_test_pred = model.predict(d_valid)
                
                curr_train_loss = log_loss(curr_y[:,i],train_pred)
                curr_val_loss = log_loss(hold_out_y[:,i],tmp_test_pred)
                
                curr_train_auc = roc_auc_score(curr_y[:,i],train_pred)
                curr_val_auc = roc_auc_score(hold_out_y[:,i],tmp_test_pred)
                
                print('ls',curr_train_loss,curr_val_loss,'auc',curr_train_auc,curr_val_auc)
                val_loss_l += curr_val_loss
                train_loss_l += curr_train_loss
                val_auc_l += curr_val_auc
                train_auc_l += curr_train_auc
            except:
                pass
            curr_test_pred = model.predict(d_test)
            test_pred[:,i] += curr_test_pred
            
            
        # avg 6 class
        train_loss_l = train_loss_l/6
        val_loss_l = val_loss_l/6
        train_auc_l = train_auc_l/6
        val_auc_l = val_auc_l/6
        print('this fold avg train',train_loss_l,'avg val',val_loss_l)
        print('this fold auc train',train_auc_l,'auc val',val_auc_l)
        
        
        # avg k fold
        all_train_loss_l += train_loss_l/k
        all_val_loss_l += val_loss_l/k
        all_train_auc_l += train_auc_l/k
        all_val_auc_l += val_auc_l/k
        print('========================')
    test_pred = test_pred/k
    print('all loss avg',all_train_loss_l,all_val_loss_l)
    print('all auc avg',all_train_auc_l,all_val_auc_l)
    print('=======================================================')
    return test_pred

print('done')

done


In [6]:
xgb_res = simple_ens('xgb',k=10)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = xgb_res
sample_submission.to_csv("../results/xgb_some_fold10.gz", index=False, compression='gzip')
print(sample_submission.head())
print('save done')
# all train avg 0.0317569652451 all val avg 0.0368260957171, PUB 9863

# all loss avg 0.0312749051012 0.0362312513023 all auc avg 0.994848895303 0.991124180632


0
ls 0.068227023931 0.0756803462098 auc 0.990956070604 0.988661955352
1
ls 0.0168721160139 0.0205382626056 auc 0.995130640603 0.992215823781
2
ls 0.0355603197706 0.0408421511733 auc 0.996117040618 0.993968340413
3
ls 0.00423023857281 0.00708573402938 auc 0.999035327784 0.997482343966
4
ls 0.0499585491875 0.0572201400685 auc 0.991664013202 0.987197869516
5
ls 0.0146531519532 0.0197015295552 auc 0.995327626385 0.988940443067
this fold avg train 0.0315835665715 avg val 0.0368446939403
this fold auc train 0.994705119866 auc val 0.991411129349
0
ls 0.0661946149474 0.0778033138322 auc 0.991723521131 0.986549293055
1
ls 0.016849369443 0.0193734259501 auc 0.995122339654 0.993433272783
2
ls 0.0351190899722 0.0356029823941 auc 0.996240371647 0.99583696848
3
ls 0.00360188875017 0.00726070374693 auc 0.99942741501 0.992413292734
4
ls 0.0494784717463 0.0491870992297 auc 0.991861987261 0.991485631165
5
ls 0.0149520654785 0.0165350506493 auc 0.995254074157 0.985675948728
this fold avg train 0.03103258